<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import numpy as np
from PIL import Image
import PIL.ImageOps
import cv2

# Takes in a path to an image, and calculates the unique bodies present in the
# Forground of the image

# str: String - Path to image

# out: 2d numpyArray - Array of unique bodies 
def segment_im(str):

    # Create our output array
    out= []

    # Read our source image
    rawImage = cv2.imread(str)
    
    # Testing
    # cv2.imshow('Original Image',rawImage)
    # cv2.waitKey(0)

    # Convert it to grey
    gray = cv2.cvtColor(rawImage,cv2.COLOR_BGR2GRAY)

	# Apply thresholding
	retval, thresholded = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
	
	# Testing
	# cv2.imshow('Thresholded Image',thresholded)
	# cv2.waitKey(0)

	# Find the unique bodies with opencv
	_, contours, hierarchy = cv2.findContours(thresholded, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

	# Create a list of contours we want to keep
	contour_list = []
	# For each contour
	for contour in contours:
	    area = cv2.contourArea(contour)
	    # If it is large enought
	    if area > 50 :
	    	# We let it in
	        contour_list.append(contour)
	
	# Draw the contours onto the source image for testing
	cv2.drawContours(rawImage, contour_list,  -1, (255,0,0), 2)
	
	# Testing
	# cv2.imshow('Objects Detected',rawImage)
	# cv2.waitKey(0)

	# For each set of contours
	for j,cnt in enumerate(contours):
		# Create a new image to paste the unique body into
		mask = np.zeros(gray.shape, np.uint8)
		# Draw it into the mask
		cv2.drawContours(mask,[cnt],0,255,-1)
		# Calculate the bounding box
		x,y,w,h = cv2.boundingRect(cnt)
		# Crop to the bounding box
		crop = mask[y:y+h,x:x+w]
		# Only export if it is large enough, but not too large
		if (cv2.countNonZero(crop) > 100) and (cv2.countNonZero(crop)<1000):
			
			# Testing
			# cv2.imshow("!!",crop)
			# cv2.waitKey(0)
			
			# Append to out array
			out.append (np.asarray(crop))

	return out


# Takes in a path to an image, and calculates the unique bodies present in the
# Forground of the image
# 
# str: String - Path to image
# 
# out: 2d numpyArray - Array of coordinates of unique bodies' bounding boxes
#						outputs in same order as segment_im
def get_bounds(str):
	
	# Create our output array
	out= []

	# Read source image
	rawImage = cv2.imread(str)
	
	# Testing
	# cv2.imshow('Original Image',rawImage)
	# cv2.waitKey(0)

	# Convert to grey
	gray = cv2.cvtColor(rawImage,cv2.COLOR_BGR2GRAY)
	
	# Apply thresholding
	retval, thresholded = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
	
	# Testing
	# cv2.imshow('Thresholded Image',thresholded)
	# cv2.waitKey(0)

	# Find contours with open cv
	_, contours, hierarchy = cv2.findContours(thresholded, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

	# For each contour
	contour_list = []
	for contour in contours:
	    area = cv2.contourArea(contour)
	    # Only let it in if it is large enough
	    if area > 50 :
	        contour_list.append(np.asarray(contour))

	# Draw contours for testing
	cv2.drawContours(rawImage, contour_list,  -1, (255,0,0), 2)
	
	# Testing
	# cv2.imshow('Objects Detected',rawImage)
	# cv2.waitKey(0)

	# For each set of contours
	for j,cnt in enumerate(contours):
		# Create a new image
		mask = np.zeros(gray.shape, np.uint8)
		# Paste the contour into it
		cv2.drawContours(mask,[cnt],0,255,-1)
		# Calculate the bounding box
		x,y,w,h = cv2.boundingRect(cnt)
		# Crop to size
		crop = mask[y:y+h,x:x+w]
		# Only let one ones which are large but not too large
		if (cv2.countNonZero(crop) > 100) and (cv2.countNonZero(crop)<1000):

			# Testing
			# cv2.imshow("!!",crop)
			# cv2.waitKey(0)

			# Save the bounding box coords
			temp = [y, y+h, x, x+w]
			# Append to out list
			out.append (temp)

	return out


# Takes in a Numpy array and returns a 
# 
# str: 2d numpyArray - Source image
# 
# out: 2d numpyArray - Flattened, resized numpy array
def resize_im(im):

		# Create the image from the array
		im = Image.fromarray(im)
		# Convert it to black and white
		im=im.convert('L')
		
		# Testing
		# im = PIL.ImageOps.invert(im)
		# im.show()

		# Get size of the image we are working on
		width, height = im.size
		# Save the larger dimention into a new variable
		newsize=max(width, height)
		# Create a new image we will paste im into that is all black
		padded_im = Image.new('RGB',
								(newsize, newsize),
								(0,0,0)
								)
		# Create the offset so that im can be centered in padded_im
		offset = (int((newsize-width)/2), int((newsize-height)/2))
		# Paste im into padded_im
		padded_im.paste(im, offset)
		padded_im = np.asarray(padded_im)
		padded_im = np.array(padded_im).copy()


		# Any pixel that is brighter than 10 can be considered white
		padded_im[padded_im>20]=255
		padded_im[padded_im<=20]=0

		padded_im = Image.fromarray(padded_im)

		# Resize the padded_im into 28x28
		padded_im = padded_im.resize((32,32))
		padded_im = padded_im.convert('1')

		# Flatten the image
		padded_im = np.asarray(padded_im).flatten()
		# Convert the array to floats ranging from 0-256
		padded_im = [float(x/256) for x in padded_im]
		return padded_im


# Reads a file from a string and returns all unique bodies after resizing
# them into a standard format
# 
# str : String - Path to image
# 
# out : 2d numpyArray of size [-1,32*32] contains all flattened, 
#		resized unique bodies
def get_segments(str):
	# Create our array with a given shape
	all_segments = np.ndarray(shape=(1,32*32))
	# Apply segmenting to get its component parts
	segments = segment_im(str)
	# For each unique body
	for s in segments:
		# Resize it
		s= resize_im(s)
		# Add it to the out array
		all_segments = np.vstack((all_segments, s))
	# When you create an ndarray with specific size it initialized the first row
	# so here we delete that first row
	all_segments= np.delete(all_segments,(0), axis=0)
	
	
	# Print for testing
	# print (all_segments)
	# print(all_segments.shape)
	
	return all_segments		
